In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler
from models.CR import CR3D
from models.CRJ import CRJ3D
from models.MCI import MCI3D
from models.ESN import ESN3D
from models.SAR import SAR3D
from models.SparseESN import SparseESN3D
from metrics.metrics import mse_dimwise, nrmse_dimwise, compute_valid_prediction_time, compute_attractor_deviation, compute_psd

In [ ]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [ ]:
def report_vpt(name, preds, test_target, time_test):
    T_VPT, T_lambda, ratio = compute_valid_prediction_time(
        test_target, preds, time_test, threshold=0.4, lambda_max=0.9
    )
    print(f"{name:20s} => T_VPT={T_VPT:.3f},  T_lambda={T_lambda:.3f}, ratio={ratio:.3f}")

In [ ]:
def main():
    # 1) Generate Lorenz data
    tmax = 110
    dt   = 0.01
    t_vals, lorenz_traj = generate_lorenz_data(
        initial_state=[1.0,1.0,1.0],
        tmax=tmax,
        dt=dt
    )

    washout = 1000
    t_vals = t_vals[washout:]
    lorenz_traj = lorenz_traj[washout:]

    # normalize
    scaler = StandardScaler()
    scaler.fit(lorenz_traj)
    lorenz_traj = scaler.transform(lorenz_traj)

    T_data = len(lorenz_traj)
    print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

    # train/test split
    train_frac = 0.7
    train_end = int(train_frac*(T_data-1))
    train_input  = lorenz_traj[:train_end]
    train_target = lorenz_traj[1:train_end+1]
    test_input   = lorenz_traj[train_end:-1]
    test_target  = lorenz_traj[train_end+1:]
    print(f"Train size: {len(train_input)}  Test size: {len(test_input)}")

    for i in [0.1, 0.2, 0.3, 0.4, 0.5]:
        mci = MCI3D(
            reservoir_size=500,
            cycle_weight=0.9,      # 'l' in the paper
            connect_weight=0.9,    # 'g' in the paper
            input_scale=0.2,
            leaking_rate=1.0,
            ridge_alpha=1e-6,
            combine_factor=0.5,    # 'h' in the paper
            seed=47,
            v1=i, v2=i
        )

        mci.fit_readout(train_input, train_target, discard=100)

        # 3) Autoregressive testing
        n_test_steps = len(test_input)
        initial_in = test_input[0]

        mci_preds = mci.predict_autoregressive(initial_in, n_test_steps)
        mse_mci    = mse_dimwise(mci_preds,    test_target)
        nrmse_mci    = nrmse_dimwise(mci_preds,    test_target)
        report_vpt("MCI",         mci_preds,    test_target, t_vals[train_end:])
        adev_mci = compute_attractor_deviation(mci_preds, test_target)

        print("\n--- Attractor Deviation ---")
        print("Minimal Comp :",  adev_mci)

        print("\n--- Multi-step Autoregressive MSE (x, y, z) ---")
        print("Minimal Comp :",  mse_mci)    
        
        print("\n--- Multi-step Autoregressive NRMSE (x, y, z) ---")
        print("Minimal Comp :",  nrmse_mci)

if __name__ == "__main__":
    main()